In [1]:
import pandas as pd 
import numpy as np
import os
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA


In [2]:
df_train = pd.read_csv('C:/Users/tun20/Desktop/Hackathons/DataQuest/dataquest-challenge-1/train.csv')
df_weather = pd.read_csv('C:/Users/tun20/Desktop/Hackathons/DataQuest/dataquest-challenge-1/weather.csv')
df_test = pd.read_csv('C:/Users/tun20/Desktop/Hackathons/DataQuest/dataquest-challenge-1/test.csv')
df_sample = pd.read_csv('C:/Users/tun20/Desktop/Hackathons/DataQuest/dataquest-challenge-1/SampleSubmission.csv')

In [3]:
df_dataset_train = pd.merge(df_train, df_weather, on= 'date', how= 'left')
df_dataset_test = pd.merge(df_test, df_weather, on= 'date', how= 'left')
df_dataset_sample = pd.merge(df_sample, df_weather, on= 'date', how= 'left')

In [4]:
df_dataset_train.describe()

,MELLEGUE,BEN METIR,KASSEB,BARBARA,SIDI SALEM,BOU-HEURTMA,JOUMINE,GHEZALA,SEJNANE,S. EL BARRAK,...,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
count,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,...,1400.000000,1346.000000,1362.000000,1284.000000,0.0,1018.000000,1209.000000,0.0,760.000000,0.0
mean,20.159376,44.669804,52.788284,134.442034,207.956659,54.388894,44.632212,19.507215,130.207464,177.044771,...,19.146000,14.037519,26.068355,0.380763,NaN,147.856582,16.269148,NaN,1016.170526,NaN
std,9.074718,11.468745,13.115188,135.148529,178.302809,18.924842,30.355657,23.187480,91.907786,114.826834,...,7.043587,6.993654,8.153006,2.621265,NaN,94.818903,6.445433,NaN,7.109143,NaN
min,4.228000,13.687000,29.528000,42.773000,23.153000,20.336000,2.618000,1.509000,34.498000,3.188000,...,4.200000,-0.200000,6.800000,0.000000,NaN,1.000000,5.200000,NaN,943.600000,NaN
25%,13.427750,38.444750,40.076000,57.355000,60.960750,40.065750,6.690750,4.868500,62.410000,8.457000,...,12.700000,7.800000,18.800000,0.000000,NaN,73.000000,11.500000,NaN,1013.300000,NaN
50%,20.767000,45.366500,55.932000,61.794500,141.088000,53.074000,48.181500,6.711000,86.828500,233.296000,...,19.200000,14.300000,25.800000,0.000000,NaN,98.500000,15.100000,NaN,1015.700000,NaN
75%,26.717000,54.442000,62.645000,126.006250,350.602250,66.037750,62.507000,34.228000,243.761500,275.649000,...,25.500000,20.425000,32.200000,0.000000,NaN,242.000000,19.800000,NaN,1019.400000,NaN
max,51.131000,63.353000,81.304000,429.730000,722.255000,120.918000,106.331000,76.519000,292.152000,307.021000,...,38.300000,29.800000,46.500000,51.800000,NaN,357.000000,46.000000,NaN,1033.700000,NaN


In [5]:
df_dataset_train['date'] = pd.to_datetime(df_dataset_train['date'], utc = True)
df_dataset_test['date'] = pd.to_datetime(df_dataset_test['date'], utc = True)
df_dataset_sample['date'] = pd.to_datetime(df_dataset_sample['date'], utc = True)
df_weather['date'] = pd.to_datetime(df_weather['date'], utc = True)


In [6]:
# Feature engineering: Extract relevant features
df_dataset_train['Month'] = df_dataset_train['date'].dt.month
df_dataset_train['Day'] = df_dataset_train['date'].dt.day
df_dataset_train['Year'] = df_dataset_train['date'].dt.year
df_dataset_test['Month'] = df_dataset_test['date'].dt.month
df_dataset_test['Day'] = df_dataset_test['date'].dt.day
df_dataset_test['Year'] = df_dataset_test['date'].dt.year
df_dataset_sample['Month'] = df_dataset_sample['date'].dt.month
df_dataset_sample['Day'] = df_dataset_sample['date'].dt.day
df_dataset_sample['Year'] = df_dataset_sample['date'].dt.year

### Model 1

In [7]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df_dataset_train, test_size=0.1, shuffle=False)

### Pycaret

In [9]:
# Melt the dataset to get the dam names in a single column
melted_train = pd.melt(df_dataset_train, id_vars=['date'], var_name='Dam', value_name='Water Level')
melted_test = pd.melt(df_dataset_test, id_vars=['date'], var_name='Dam', value_name='Water Level')
melted_sample = pd.melt(df_dataset_sample, id_vars=['date'], var_name='Dam', value_name='Water Level')

In [10]:
train_data = pd.merge(melted_train, df_weather, on='date', how='left')
test_data = pd.merge(melted_test, df_weather, on='date', how='left')
sample_data = pd.merge(melted_sample, df_weather, on='date', how='left')

In [11]:

from pycaret.regression import *

# Set up pycaret regressor 
exp_reg102 = setup(data = train_data, target = 'Water Level', session_id=69, normalize=True, train_size=0.9, fold=3,use_gpu = False) 

,Description,Value
0,session_id,69
1,Target,Water Level
2,Original Data,"(57400, 13)"
3,Missing Values,True
4,Numeric Features,7
5,Categorical Features,4
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(45277, 66)"


In [12]:
# Filter best models 

best = compare_models(sort='R2', n_select = 4)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,16.7190,1777.0917,42.1435,0.9857,0.8571,61.3622,0.7767
rf,Random Forest Regressor,15.4766,1834.1687,42.8107,0.9852,0.7592,46.9361,11.9133
xgboost,Extreme Gradient Boosting,16.3814,1856.2054,43.0763,0.9851,0.8200,54.1223,4.0133
et,Extra Trees Regressor,16.2825,1986.2761,44.5519,0.9840,0.7833,48.3508,9.3733
gbr,Gradient Boosting Regressor,21.9337,2032.8873,45.0799,0.9836,1.2293,80.6534,2.6233
ridge,Ridge Regression,21.0990,2291.5479,47.8648,0.9816,0.9459,62.0847,1.3467
br,Bayesian Ridge,21.0825,2291.3538,47.8628,0.9816,0.9445,61.6123,0.7633
lar,Least Angle Regression,21.1868,2293.5515,47.8855,0.9815,0.9525,61.4104,0.1767
lr,Linear Regression,21.7458,2309.1837,48.0504,0.9814,1.0033,61.1916,3.8833
huber,Huber Regressor,18.4080,2634.4729,51.3115,0.9788,0.8314,55.1873,3.9333


In [13]:
# Initialize best model 

rf = create_model('rf', verbose=True, return_train_score= True)

In [12]:
# Tune best model

tuned_rf = tune_model(rf, optimize = 'MAE', choose_better = True, verbose= True, early_stopping=True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,20.4334,1800.3235,42.4302,0.9854,1.1938,56.8837
1,20.6740,1988.9870,44.5981,0.9841,1.2016,66.2756
2,20.1154,1817.6406,42.6338,0.9854,1.1983,52.8832
Mean,20.4076,1868.9837,43.2207,0.9850,1.1979,58.6808
Std,0.2287,85.1491,0.9775,0.0006,0.0032,5.6132


In [13]:
evaluate_model(tuned_rf)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [14]:
sample_data.drop(columns={'Water Level'}, inplace= True)

In [16]:
predict_model(rf, data= sample_data)

,date,Dam,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,Label
0,2019-02-06 00:00:00+00:00,MELLEGUE,9.5,2.8,15.8,0.0,NaN,266.0,14.1,NaN,1019.5,NaN,26.25062
1,2019-02-07 00:00:00+00:00,MELLEGUE,9.5,2.8,15.8,0.0,NaN,253.0,11.0,NaN,1021.1,NaN,21.97921
2,2019-02-08 00:00:00+00:00,MELLEGUE,10.2,2.8,17.8,NaN,NaN,255.0,14.0,NaN,1022.3,NaN,25.56322
3,2019-02-09 00:00:00+00:00,MELLEGUE,10.8,3.8,18.8,0.0,NaN,279.0,8.7,NaN,1024.4,NaN,20.69714
4,2019-02-11 00:00:00+00:00,MELLEGUE,13.9,9.8,20.8,0.0,NaN,235.0,22.8,NaN,1021.4,NaN,11.57513
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15001,2020-10-01 00:00:00+00:00,Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.66000
15002,2020-10-02 00:00:00+00:00,Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.55000
15003,2020-11-01 00:00:00+00:00,Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.54000
15004,2020-11-02 00:00:00+00:00,Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.49000


In [17]:
prediction = predict_model(rf, data= sample_data)

In [18]:

df = prediction.pivot(index='date', columns='Dam', values='Label')

In [19]:
df_dataset_sample.columns

Index(['date', 'MELLEGUE', 'BEN METIR', 'KASSEB', 'BARBARA', 'SIDI SALEM',
       'BOU-HEURTMA', 'JOUMINE', 'GHEZALA', 'SEJNANE', 'S. EL BARRAK',
       'SILIANA', 'LAKHMESS', 'RMIL', 'BIR M'CHERGA', 'RMEL', 'NEBHANA',
       'SIDI SAAD', 'EL HAOUAREB', 'SIDI AÏCH', 'EL BREK', 'BEZIRK', 'CHIBA',
       'MASRI', 'LEBNA', 'HMA', 'ABID', 'Zarga', 'Ziatine', 'tavg', 'tmin',
       'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'Month',
       'Day', 'Year'],
      dtype='object')

In [20]:
df.columns

Index(['ABID', 'BARBARA', 'BEN METIR', 'BEZIRK', 'BIR M'CHERGA', 'BOU-HEURTMA',
       'CHIBA', 'Day', 'EL BREK', 'EL HAOUAREB', 'GHEZALA', 'HMA', 'JOUMINE',
       'KASSEB', 'LAKHMESS', 'LEBNA', 'MASRI', 'MELLEGUE', 'Month', 'NEBHANA',
       'RMEL', 'RMIL', 'S. EL BARRAK', 'SEJNANE', 'SIDI AÏCH', 'SIDI SAAD',
       'SIDI SALEM', 'SILIANA', 'Year', 'Zarga', 'Ziatine', 'prcp', 'pres',
       'snow', 'tavg', 'tmax', 'tmin', 'tsun', 'wdir', 'wpgt', 'wspd'],
      dtype='object', name='Dam')

In [21]:
df.drop(columns={'Day', 'Month', 'Year','prcp', 'pres',
       'snow', 'tavg', 'tmax', 'tmin', 'tsun', 'wdir', 'wpgt', 'wspd'}, inplace= True)

In [22]:
df = df[['MELLEGUE', 'BEN METIR', 'KASSEB', 'BARBARA', 'SIDI SALEM',
       'BOU-HEURTMA', 'JOUMINE', 'GHEZALA', 'SEJNANE', 'S. EL BARRAK',
       'SILIANA', 'LAKHMESS', 'RMIL', "BIR M'CHERGA", 'RMEL', 'NEBHANA',
       'SIDI SAAD', 'EL HAOUAREB', 'SIDI AÏCH', 'EL BREK', 'BEZIRK', 'CHIBA',
       'MASRI', 'LEBNA', 'HMA', 'ABID', 'Zarga', 'Ziatine']]

In [26]:
df.reset_index(inplace= True)

In [29]:
# Format the date column with the desired format
df['date'] = df['date'].dt.strftime('%Y-%m-%d')

In [30]:
df.to_csv('sample1.csv', index= False)